In [1]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# Inicializar un navegador web Chrome a través de WebDriver
driver = webdriver.Chrome()

# Navegar a la página web de AEMET que contiene información del tiempo para municipios
driver.get("https://www.aemet.es/es/eltiempo/prediccion/municipios?p=28")

# Esperar 3 segundos para que la página cargue completamente
sleep(3)

# Crear un diccionario llamado "resultados_aemet" para almacenar la información del tiempo
resultados_aemet = {
    "municipio": [],
    "temperatura": [],
    "fecha": []
}

# Encontrar el elemento select donde se selecciona el municipio
select = driver.find_element("css selector", "#localidades_selector")

# Encontrar todas las opciones dentro del elemento select
opcion = select.find_elements("tag name", "option")

# Esperar 3 segundos
sleep(3)

# Iterar a través de las primeras 4 opciones (excluyendo la primera que es un encabezado)
for valores in opcion[1:5]:
    # Imprimir el valor del atributo "value" de cada opción (generalmente contiene el código del municipio)
    print(valores.get_attribute("value"))

    # Obtener el nombre del municipio
    municipio = valores.text

    # Esperar 5 segundos
    sleep(5)

    # Seleccionar la opción actual en el elemento select
    seleccionar = Select(driver.find_element("css selector", "#localidades_selector"))
    seleccionar.select_by_value(valores.get_attribute("value"))

    # Hacer clic para entrar en los datos de este municipio
    driver.find_element("css selector", "#contenedor > div > div.contenedor_contenido > div:nth-child(4) > form > div.contenedor_central > div > input").click()

    # Esperar 5 segundos
    sleep(5)

    # Extraer la fecha de los datos del tiempo
    resultados_aemet["fecha"].append(driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div[2]/div[3]/div[3]/table/thead/tr').text)

    # Extraer la temperatura de los datos del tiempo
    resultados_aemet["temperatura"].append(driver.find_element("css selector", "#tabla_prediccion > tbody > tr:nth-child(7)").text)

    # Agregar el nombre del municipio al diccionario "resultados_aemet"
    resultados_aemet["municipio"].append(municipio)

    # Esperar 5 segundos
    sleep(5)
    
    # Ir a la página anterior (la lista de municipios)
    driver.back()

    # Esperar 2 segundos
    sleep(2)

# Cerrar el navegador
driver.close()

acebeda-la-id28001
ajalvir-id28002
alameda-del-valle-id28003
alamo-el-id28004


In [3]:
# almacenar los resultados en un DataFrame
df_aemet = pd.DataFrame(resultados_aemet)

# mostrar los resultados almacenados en el DataFrame
df_aemet

,municipio,temperatura,fecha
0,"Acebeda, La",-2 / 12 -4 / 11 -3 / 13 2 / 12 2 / 14 3 /...,lun. 22 mar. 23 mié. 24 jue. 25 vie. 26 sáb. 2...
1,Ajalvir,5 / 19 1 / 18 3 / 19 8 / 18 7 / 21 8 / 18...,lun. 22 mar. 23 mié. 24 jue. 25 vie. 26 sáb. 2...
2,Alameda del Valle,-3 / 15 -5 / 13 -3 / 16 3 / 15 2 / 17 3 /...,lun. 22 mar. 23 mié. 24 jue. 25 vie. 26 sáb. 2...
3,"Álamo, El",6 / 21 3 / 19 4 / 20 9 / 20 8 / 21 8 / 18...,lun. 22 mar. 23 mié. 24 jue. 25 vie. 26 sáb. 2...
